# doc2vecを用いた単語の特徴ベクトル学習 (テストラン: 1回反復)

In [1]:
import gensim
import os
import collections
import random
import csv
import sys

In [2]:
csv.field_size_limit(sys.maxsize)

131072

In [3]:
corpusfile = 'novel2vec.txt'
modelfile = 'novel2vec.model'

In [4]:
def read_corpus():
    with open(corpusfile, newline='', encoding='utf-8') as csvfile:
        novels = csv.reader(csvfile)
        for novel in novels:
            novel_id, tokens_text = novel
            tokens = gensim.utils.to_unicode(tokens_text).split()
            yield gensim.models.doc2vec.TaggedDocument(tokens, [novel_id])

In [5]:
corpus = list(read_corpus())

In [6]:
model = gensim.models.doc2vec.Doc2Vec(size=100, min_count=5, iter=1, workers=36)

In [7]:
model.build_vocab(corpus)

In [8]:
import resource
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

32802984

In [9]:
%time model.train(corpus)

CPU times: user 56min 29s, sys: 2.25 s, total: 56min 31s
Wall time: 3min 28s


317853348

In [10]:
model.save('novel2vec-iter1.model')

In [10]:
ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(corpus[doc_id].tags[0])
    ranks.append(rank)
    second_ranks.append(sims[1])

In [11]:
collections.Counter(ranks)

Counter({0: 995, 1: 1, 3: 4})